# Workshop 2: Human-in-the-Loop (HITL)

In this workshop, you'll learn how to integrate human oversight into an AI agent workflow using Telegram as the notification channel.

## What You'll Build

1. **Send a simple message** to Telegram from Python
2. **Add interactive buttons** (Approve/Reject)
3. **Build an approval workflow** that waits for human decision
4. **Capture feedback** when posts are rejected

## Prerequisites

- Telegram account
- A Telegram bot (we'll create one together)
- Python environment with `python-telegram-bot` installed

---
## Step 0: Setup Your Telegram Bot

### Create a Bot with @BotFather

1. Open Telegram and search for `@BotFather`
2. Send `/newbot`
3. Choose a name (e.g., "My HITL Bot")
4. Choose a username (must end in `bot`, e.g., `my_hitl_workshop_bot`)
5. **Save the token** you receive

### Get Your Chat ID

1. Start a chat with your new bot (send any message)
2. Visit: `https://api.telegram.org/bot<YOUR_TOKEN>/getUpdates`
3. Find `"chat":{"id":XXXXXXXX}` in the response
4. **Save this chat ID**

---
## Step 1: Configure Your Environment Variables

Fill in your credentials below. **Never commit these to git!**

In [2]:
import os

# ============================================
# TODO: Replace with your actual credentials
# ============================================
os.environ["TELEGRAM_BOT_TOKEN"] = "8367469580:AAH6Is_YQb76dfnYuMpyLOG7KGWUrSD0uHk"
os.environ["TELEGRAM_CHAT_ID"] = "6281060305"

# Verify they're set
print("Bot token set:", "TELEGRAM_BOT_TOKEN" in os.environ)
print("Chat ID set:", "TELEGRAM_CHAT_ID" in os.environ)

Bot token set: True
Chat ID set: True


---
## Step 2: Send a Simple Message

Let's start with the basics: sending a text message to Telegram.

In [3]:
# !pip install python-telegram-bot

In [4]:
import asyncio
from telegram import Bot

async def send_simple_message(text: str):
    """Send a basic text message to Telegram."""
    bot = Bot(token=os.environ["TELEGRAM_BOT_TOKEN"])
    message = await bot.send_message(
        chat_id=int(os.environ["TELEGRAM_CHAT_ID"]),
        text=text,
    )
    print(f"✅ Message sent! ID: {message.message_id}")
    return message

# Run it!
await send_simple_message("👋 Hello from Python! This is my first HITL message.")

✅ Message sent! ID: 3


Message(channel_chat_created=False, chat=Chat(first_name='Anon', id=6281060305, last_name='Person', type=<ChatType.PRIVATE>), date=datetime.datetime(2026, 1, 21, 20, 48, 12, tzinfo=datetime.timezone.utc), delete_chat_photo=False, from_user=User(first_name='My HITL Bot', id=8367469580, is_bot=True, username='ewfesfefwefe_bot'), group_chat_created=False, message_id=3, supergroup_chat_created=False, text='👋 Hello from Python! This is my first HITL message.')

### ✏️ Exercise 2.1

Modify the message text and run the cell again. Try:
- Adding emojis
- Sending a longer message
- Sending multiple messages in sequence

In [5]:
# Your code here - experiment with different messages!
await send_simple_message("Hello please send nudes 🍑")

✅ Message sent! ID: 4


Message(channel_chat_created=False, chat=Chat(first_name='Anon', id=6281060305, last_name='Person', type=<ChatType.PRIVATE>), date=datetime.datetime(2026, 1, 21, 20, 49, 16, tzinfo=datetime.timezone.utc), delete_chat_photo=False, from_user=User(first_name='My HITL Bot', id=8367469580, is_bot=True, username='ewfesfefwefe_bot'), group_chat_created=False, message_id=4, supergroup_chat_created=False, text='Hello please send nudes 🍑')

---
## Step 3: Add Interactive Buttons

Now let's add **inline keyboard buttons** so the human can respond with a single tap.

In [6]:
from telegram import InlineKeyboardButton, InlineKeyboardMarkup

async def send_message_with_buttons(text: str):
    """Send a message with Approve/Reject buttons."""
    bot = Bot(token=os.environ["TELEGRAM_BOT_TOKEN"])

    # Create the keyboard with two buttons
    keyboard = InlineKeyboardMarkup([
        [
            InlineKeyboardButton("✅ Approve", callback_data="approve"),
            InlineKeyboardButton("❌ Reject", callback_data="reject"),
        ]
    ])

    message = await bot.send_message(
        chat_id=int(os.environ["TELEGRAM_CHAT_ID"]),
        text=text,
        reply_markup=keyboard,
    )
    print(f"✅ Message with buttons sent! ID: {message.message_id}")
    return message

# Run it!
await send_message_with_buttons("📝 New Post for Review\n\nThis is a sample post. Do you approve?")

✅ Message with buttons sent! ID: 5


Message(channel_chat_created=False, chat=Chat(first_name='Anon', id=6281060305, last_name='Person', type=<ChatType.PRIVATE>), date=datetime.datetime(2026, 1, 21, 20, 50, 12, tzinfo=datetime.timezone.utc), delete_chat_photo=False, from_user=User(first_name='My HITL Bot', id=8367469580, is_bot=True, username='ewfesfefwefe_bot'), group_chat_created=False, message_id=5, reply_markup=InlineKeyboardMarkup(inline_keyboard=((InlineKeyboardButton(callback_data='approve', text='✅ Approve'), InlineKeyboardButton(callback_data='reject', text='❌ Reject')),)), supergroup_chat_created=False, text='📝 New Post for Review\n\nThis is a sample post. Do you approve?')

**Note:** The buttons appear but don't do anything yet! We need to set up a listener to handle the button clicks.

### ✏️ Exercise 3.1

Modify the buttons to have different labels or add more options:

In [7]:
async def send_custom_buttons():
    """Experiment with different button layouts."""
    bot = Bot(token=os.environ["TELEGRAM_BOT_TOKEN"])

    # TODO: Try different button configurations!
    # Example: Add a third "Edit" button, or arrange buttons vertically
    keyboard = InlineKeyboardMarkup([
        [
            InlineKeyboardButton("👍 Like", callback_data="like"),
            InlineKeyboardButton("👎 Dislike", callback_data="dislike"),
        ],
        # Add more rows here for vertical arrangement
        [
            InlineKeyboardButton("🔄 Regenerate", callback_data="regenerate"),
            InlineKeyboardButton("🗑️ Trash", callback_data="delete"),
        ]
    ])

    await bot.send_message(
        chat_id=int(os.environ["TELEGRAM_CHAT_ID"]),
        text="Rate this content:",
        reply_markup=keyboard,
    )

await send_custom_buttons()

---
## Step 4: Handle Button Clicks (Polling)

To respond to button clicks, we need to run a **polling loop** that listens for updates from Telegram.

**Important:** This cell will run until you stop it (Kernel > Interrupt).

In [8]:
from telegram.ext import Application, CallbackQueryHandler, ContextTypes
from telegram import Update

async def handle_button_click(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handle button clicks from Telegram."""
    query = update.callback_query
    await query.answer()  # Acknowledge the click

    action = query.data
    print(f"📥 Button clicked: {action}")

    if action == "approve":
        await query.edit_message_text("✅ APPROVED! Post will be published.")
    elif action == "reject":
        await query.edit_message_text("❌ REJECTED. Post discarded.")
    else:
        await query.edit_message_text(f"Received: {action}")

# Build and run the application
print("🚀 Starting bot... Click a button in Telegram!")
print("⏹️  Press the Stop button in Jupyter to stop.\n")

app = Application.builder().token(os.environ["TELEGRAM_BOT_TOKEN"]).build()
app.add_handler(CallbackQueryHandler(handle_button_click))

# Run polling (this blocks until interrupted)
await app.initialize()
await app.start()
await app.updater.start_polling()

# Keep running until interrupted
try:
    await asyncio.Event().wait()
except asyncio.CancelledError:
    pass
finally:
    await app.updater.stop()
    await app.stop()
    await app.shutdown()
    print("\n🛑 Bot stopped.")

🚀 Starting bot... Click a button in Telegram!
⏹️  Press the Stop button in Jupyter to stop.

📥 Button clicked: like
📥 Button clicked: dislike
📥 Button clicked: approve

🛑 Bot stopped.


---
## Step 5: Complete Approval Workflow

Now let's build a complete workflow that:
1. Sends a post for approval
2. Waits for human decision
3. Returns the result

This is the core HITL pattern!

In [9]:
# Global state for the approval flow
pending_post = None
decision_result = None
decision_made = asyncio.Event()

async def wait_for_approval(post_content: str) -> str:
    """
    Send a post for approval and wait for human decision.
    Returns 'approve' or 'reject'.
    """
    global pending_post, decision_result

    async def handle_decision(update: Update, context: ContextTypes.DEFAULT_TYPE):
        global decision_result
        query = update.callback_query
        await query.answer()

        decision_result = query.data
        if decision_result == "approve":
            await query.edit_message_text(f"✅ APPROVED\n\n{pending_post}")
        else:
            await query.edit_message_text(f"❌ REJECTED\n\n{pending_post[:100]}...")

        decision_made.set()

    # Reset state
    pending_post = post_content
    decision_result = None
    decision_made.clear()

    # Send the post with buttons
    bot = Bot(token=os.environ["TELEGRAM_BOT_TOKEN"])
    keyboard = InlineKeyboardMarkup([
        [
            InlineKeyboardButton("✅ Approve", callback_data="approve"),
            InlineKeyboardButton("❌ Reject", callback_data="reject"),
        ]
    ])

    await bot.send_message(
        chat_id=int(os.environ["TELEGRAM_CHAT_ID"]),
        text=f"📝 New Post for Approval\n\n{post_content}\n\nCharacters: {len(post_content)}",
        reply_markup=keyboard,
    )
    print("📱 Sent to Telegram. Waiting for approval...")

    # Set up the listener
    app = Application.builder().token(os.environ["TELEGRAM_BOT_TOKEN"]).build()
    app.add_handler(CallbackQueryHandler(handle_decision))

    await app.initialize()
    await app.start()
    await app.updater.start_polling()

    # Wait for decision
    await decision_made.wait()

    # Cleanup
    await app.updater.stop()
    await app.stop()
    await app.shutdown()

    return decision_result

In [10]:
# Test the approval workflow
sample_post = """🚀 Excited to share our latest AI insights!

We've been exploring how LLMs can help automate content creation while keeping humans in control.

#AI #Automation #HITL"""

decision = await wait_for_approval(sample_post)

if decision == "approve":
    print("\n✅ Post approved! Ready to publish.")
    # Here you would call your publishing function
    # e.g., post_to_mastodon(sample_post)
else:
    print("\n❌ Post rejected. Not published.")

📱 Sent to Telegram. Waiting for approval...

✅ Post approved! Ready to publish.


---
## Step 6: Capture Rejection Feedback

A simple Approve/Reject is useful, but **why** was it rejected? Let's capture that feedback!

This is the key insight: the more information we capture, the more we can improve the system.

In [11]:
from telegram.ext import MessageHandler, filters

# State for feedback flow
feedback_pending_post = None
feedback_decision = None
feedback_reason = None
waiting_for_reason = False
feedback_done = asyncio.Event()

async def wait_for_approval_with_feedback(post_content: str) -> tuple[str, str | None]:
    """
    Send post for approval. If rejected, collect the reason.
    Returns (decision, rejection_reason).
    """
    global feedback_pending_post, feedback_decision, feedback_reason, waiting_for_reason

    async def handle_button(update: Update, context: ContextTypes.DEFAULT_TYPE):
        global feedback_decision, waiting_for_reason
        query = update.callback_query
        await query.answer()

        if query.data == "approve":
            feedback_decision = "approve"
            await query.edit_message_text(f"✅ APPROVED\n\n{feedback_pending_post}")
            feedback_done.set()
        elif query.data == "reject":
            feedback_decision = "reject"
            waiting_for_reason = True
            await query.edit_message_text(
                "❌ REJECTED\n\n"
                "Please reply with the reason for rejection.\n"
                "This feedback helps improve future posts.\n\n"
                "Examples: 'Too promotional' or 'Wrong tone'"
            )

    async def handle_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
        global feedback_reason, waiting_for_reason
        if not waiting_for_reason:
            return

        feedback_reason = update.message.text
        waiting_for_reason = False
        await update.message.reply_text(
            f"📝 Feedback recorded!\n\nReason: {feedback_reason}"
        )
        feedback_done.set()

    # Reset state
    feedback_pending_post = post_content
    feedback_decision = None
    feedback_reason = None
    waiting_for_reason = False
    feedback_done.clear()

    # Send the post
    bot = Bot(token=os.environ["TELEGRAM_BOT_TOKEN"])
    keyboard = InlineKeyboardMarkup([
        [
            InlineKeyboardButton("✅ Approve", callback_data="approve"),
            InlineKeyboardButton("❌ Reject", callback_data="reject"),
        ]
    ])

    await bot.send_message(
        chat_id=int(os.environ["TELEGRAM_CHAT_ID"]),
        text=f"📝 New Post for Approval\n\n{post_content}\n\nCharacters: {len(post_content)}",
        reply_markup=keyboard,
    )
    print("📱 Sent to Telegram. Waiting for approval...")

    # Set up listeners
    app = Application.builder().token(os.environ["TELEGRAM_BOT_TOKEN"]).build()
    app.add_handler(CallbackQueryHandler(handle_button))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text))

    await app.initialize()
    await app.start()
    await app.updater.start_polling()

    # Wait for completion
    await feedback_done.wait()

    # Cleanup
    await app.updater.stop()
    await app.stop()
    await app.shutdown()

    return feedback_decision, feedback_reason

In [12]:
# Test the feedback workflow
sample_post = """🔥 BUY NOW! Our AMAZING AI product is the BEST EVER!!!

Limited time offer! Don't miss out! Act fast!

#AI #BuyNow #Amazing"""

decision, reason = await wait_for_approval_with_feedback(sample_post)

print(f"\n📊 Result:")
print(f"   Decision: {decision}")
print(f"   Reason: {reason}")

if decision == "reject" and reason:
    print(f"\n💡 Feedback to improve the prompt:")
    print(f"   The human said: '{reason}'")
    print(f"   Consider adjusting your prompt to avoid this issue.")

📱 Sent to Telegram. Waiting for approval...

📊 Result:
   Decision: reject
   Reason: I don't like using emojis

💡 Feedback to improve the prompt:
   The human said: 'I don't like using emojis'
   Consider adjusting your prompt to avoid this issue.


---
## Step 7: Putting It All Together

Now you have all the building blocks! Here's how they fit into a complete agent workflow:

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  Generate   │ ──► │   Review    │ ──► │   Human     │ ──► │   Publish   │
│    Post     │     │  (Telegram) │     │  Decision   │     │  or Discard │
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
                                              │
                                              ▼
                                        ┌─────────────┐
                                        │  Feedback   │
                                        │   (if ❌)   │
                                        └─────────────┘
                                              │
                                              ▼
                                        ┌─────────────┐
                                        │  Improve    │
                                        │   Prompt    │
                                        └─────────────┘
```

---
## 🎯 Exercises

### Exercise A: Add an "Edit" Option
Modify the approval workflow to include a third button: "✏️ Edit". When clicked, allow the human to send a revised version of the post.

### Exercise B: Store Feedback
Create a simple list or file that stores all rejection reasons. Use this data to show patterns (e.g., "Posts rejected 3 times for 'too promotional'").

### Exercise C: Connect to Workshop 1
Import the `generate_post` function from Workshop 1 and create a complete pipeline:
1. Generate a post with the LLM
2. Send for human approval
3. If approved, publish to Mastodon
4. If rejected, store the feedback

In [ ]:
# Exercise A: Your code here
# Hint: Add a third button and handle the "edit" callback_data


In [ ]:
# Exercise B: Your code here
# Hint: Create a list to store feedback and print a summary
feedback_history = []


In [ ]:
# Exercise C: Your code here
# Hint: Import from workshop-1 and connect the pieces
# import sys
# sys.path.insert(0, "../workshop-1")
# from post_generator import generate_post, post_to_mastodon


---
## 🔑 Key Takeaways

1. **HITL is strategic** - Place checkpoints where human judgment adds unique value
2. **Feedback richness matters** - Approve/Reject < Reason < Edit
3. **Memory closes the loop** - Store and use feedback to improve over time
4. **Meet humans where they are** - Telegram, Slack, email... the channel matters less than the workflow

## Next Steps

In **Workshop 3**, we'll explore how to:
- Store feedback in a database
- Use past feedback to improve prompts automatically
- Build memory into your agent